# Example usage of GraphQL API in python.
This notebook demonstrates the usage of the `createMany` endpoint and is used to import the `4pp.csv` dataset into the database.<br/>
If you'd like to explore the GraphQL API and it's schema in detail go to the [GraphQL Explorer](http://localhost:4000), the `example.ipynb` notebook shows how to query data from the API.

In [ ]:
!pip install gql[all]

In [ ]:
from gql import gql, Client
from gql.dsl import dsl_gql, DSLSchema, DSLQuery
from gql.transport.requests import RequestsHTTPTransport

import csv

In [ ]:
# Select your transport with a defined url endpoint
transport = RequestsHTTPTransport(url="http://localhost:4000/")

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)

### Batch ingesting data


In [ ]:
data_path = "./data/4pp.csv"

In [ ]:
data = []
with open(data_path) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append({
            "woonplaats": str(row["woonplaats"]),
            "soort": str(row["soort"]),
            "provincie": str(row["provincie"]),
            "postcode": int(row["postcode"]),
            "netnummer": str(row["netnummer"]),
            "longitude": float(row["longitude"]),
            "latitude": float(row["latitude"]),
            "gemeente": str(row["gemeente"]),
            "alternatieve_schrijfwijzen": str(row["alternatieve_schrijfwijzen"]),
        })

In [ ]:
# Provide the createMany graphql query.
# Queries can be found using the GraphQL explorer at http://localhost:4000/
query = gql("""
  mutation Mutation($data: [ZipcodeCreateInput!]) {
    createManyZipcode(data: $data) {
      count
    }
  }
""")

# Execute the query using the graphql client.
# The $data variable in the gql query gets populated with the data from the `variable_values` parameter below.
result = client.execute(query, variable_values={
  "data": data
})

count = result['createManyZipcode']['count']

In [ ]:
print(f"""
Rows in CSV file: {len(data)}
Rows added to database: {count}
""")